In [1]:


!pip install librosa soundfile tensorflow --quiet

import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")


def extract_features(file_path, sr=16000, max_len=3, n_mfcc=40):
    """
    تقرأ ملف صوت وتحوّله إلى مصفوفة MFCC ثابتة الطول
    max_len = عدد الثواني (نقص أو نطوّل الصوت لهذا الطول)
    """
    y, sr = librosa.load(file_path, sr=sr)
    max_samples = max_len * sr

    # قص أو حشو الصوت لطول ثابت
    if len(y) > max_samples:
        y = y[:max_samples]
    else:
        y = np.pad(y, (0, max_samples - len(y)))

    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc = mfcc.T   # الشكل (time, features)
    return mfcc




file_paths = {
    "gunshot": [
        "gunshot.m4a"
    ],
    "help": [
        "help.m4a",
        "تسجيل2025-12-09 18-03-35.m4a",
        "تسجيل2025-12-09 18-03-50.m4a",
        "تسجيل2025-12-09 18-04-30.m4a"
    ],
    "normal": [
        "normal.m4a"
    ]
}

X_list = []
y_list = []

for label, paths in file_paths.items():
    for path in paths:
        try:
            feats = extract_features(path)
            X_list.append(feats)
            y_list.append(label)
            print(f"✔️ تم إدخال ملف {path} ضمن فئة {label}")
        except Exception as e:
            print(f"❌ خطأ في {path}: {e}")

X = np.array(X_list)
y = np.array(y_list)

print("\nعدد العينات الكلي:", len(X))
print("شكل العينة الواحدة:", X[0].shape)

# تحويل الشكل ليوافق مودل CNN: (عينات, وقت, خصائص, قناة)
X = X[..., np.newaxis]

# ترميز الليبلات إلى أرقام
le = LabelEncoder()
y_enc = le.fit_transform(y)

print("الليبلات:", list(zip(y, y_enc)))



input_shape = X.shape[1:]

model = models.Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(le.classes_), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print("\nملخص النموذج:")
print(model.summary())



repeat_times = 40  # كرري العينات للتدريب (للديمو فقط)

X_train = np.repeat(X, repeat_times, axis=0)
y_train = np.repeat(y_enc, repeat_times, axis=0)

history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=8,
    verbose=1
)

print("\nتم تدريب النموذج بنجاح ✅")



def predict_audio(path):
    print(f"\nتجربة الملف: {path}")
    feats = extract_features(path)
    feats = feats[np.newaxis, ..., np.newaxis]  # (1, time, features, channel)
    probs = model.predict(feats)[0]
    idx = np.argmax(probs)
    label = le.inverse_transform([idx])[0]
    print("🔊 التوقع:", label)
    print("نِسَب الاحتمال لكل فئة:")
    for cls, p in zip(le.classes_, probs):
        print(f" - {cls}: {p:.3f}")



print("\n====== تجربة على العينات الأصلية ======")
predict_audio("gunshot.m4a")
predict_audio("help.m4a")
predict_audio("normal.m4a")

# يمكن أيضًا تجربة التسجيلات الجديدة لـ "ساعدوني" مثلاً:
# predict_audio("تسجيل2025-12-09 18-03-35.m4a")
# predict_audio("تسجيل2025-12-09 18-03-50.m4a")
# predict_audio("تسجيل2025-12-09 18-04-30.m4a")


✔️ تم إدخال ملف gunshot.m4a ضمن فئة gunshot
✔️ تم إدخال ملف help.m4a ضمن فئة help
✔️ تم إدخال ملف تسجيل2025-12-09 18-03-35.m4a ضمن فئة help
✔️ تم إدخال ملف تسجيل2025-12-09 18-03-50.m4a ضمن فئة help
✔️ تم إدخال ملف تسجيل2025-12-09 18-04-30.m4a ضمن فئة help
✔️ تم إدخال ملف normal.m4a ضمن فئة normal

عدد العينات الكلي: 6
شكل العينة الواحدة: (94, 40)
الليبلات: [(np.str_('gunshot'), np.int64(0)), (np.str_('help'), np.int64(1)), (np.str_('help'), np.int64(1)), (np.str_('help'), np.int64(1)), (np.str_('help'), np.int64(1)), (np.str_('normal'), np.int64(2))]

ملخص النموذج:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 92, 38, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 46, 19, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 44, 17, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 22, 8, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 5632)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       360,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 365,507 (1.39 MB)

 Trainable params: 365,507 (1.39 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.6950 - loss: 7.6288
Epoch 2/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 1.0000 - loss: 3.9971e-08
Epoch 3/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 4/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 5/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 6/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 7/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 8/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 9/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 10/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 11/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 12/20
30/30 

# New Section

In [2]:
predict_file("help.csv")

NameError: name 'predict_file' is not defined